# url
We can further refine and enhance the extracted information by modifying our approach or extracting additional details from the URL.

In [20]:
import requests
from bs4 import BeautifulSoup
import re

# URL to scrape
url = 'https://www.gray-point.com/properties/'

# Add headers to avoid getting blocked
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Fetch and parse the page
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract property details
properties = []
for listing in soup.find_all('div', class_='PropertyContent'):
    name = listing.find('h1').get_text(strip=True) if listing.find('h1') else "N/A"

    # Extract all <p> tags and join their text content
    paragraphs = [p.get_text(strip=True) for p in listing.find_all('p')]
    address_info = " | ".join(paragraphs) if paragraphs else "N/A"

    # Split the extracted content at "|"
    split_info = address_info.split(" | ")

    # Extract Price (5th row of "Other Info" or match currency patterns)
    price = "N/A"
    if len(split_info) > 2:
        price = split_info[2]  # Extract the 5th row
    else:
        # Try to find price patterns like £500, £1,390,000, etc.
        price_match = re.search(r'£[\d,]+(?:\.\d+)?', address_info)
        if price_match:
            price = price_match.group()

    properties.append({
        'Name': name,
        'Address': split_info[0] if len(split_info) > 0 else "N/A",
        'Other Info': "\n".join(split_info[1:]) if len(split_info) > 1 else "N/A",
        'Price': price  # New column
    })

# Print extracted data in a structured format
if properties:
    for prop in properties:
        print(f"Name: {prop['Name']}")
        print(f"Address: {prop['Address']}")
        print("Other Info:")
        print(prop['Other Info'])
        print(f"Price: {prop['Price']}")  # Display the price
        print("-" * 50)
else:
    print("No properties found. Check the HTML structure.")


Name: Sovereign Gate, 18-20 Kew Road, Richmond TW9 2NA
Address: OFFICE TO LET RICHMOND / SERVICED OFFICE DESKS
Other Info:
Office Serviced Desks to let Richmond
Desk rates from £485 – £500 per desk
Net Internal Area: 700 sq ft – 39,135 sq ft approx.
Download property information flyer
Price: Desk rates from £485 – £500 per desk
--------------------------------------------------
Name: 179 High Street, Hampton Hill TW12 1NL
Address: NOW LET TO ESTABLISHED SUPERMARKET CHAIN
Other Info:
Hampton Hill, London
Gross Internal Area: 4,209 sq. ft. (391 sq. m) approx.
Download property information flyer
Price: Gross Internal Area: 4,209 sq. ft. (391 sq. m) approx.
--------------------------------------------------
Name: 28 York Street, Twickenham TW1 3LJ
Address: ﻿
Other Info:
FREEHOLD (3 new flats & 1 shop)
Unconditional Offers of £1,390,000
Freehold for Sale Twickenham
Shop for Sale Twickenham
Commercial Property for Sale Twickenham
Residential for Sale Twickenham
Commercial space: 969 Sq Ft / 

In [21]:
df1 = pd.DataFrame(properties)
# excel_file_path = r"C:\Users\HP\Downloads\Aadhaar_Extracted_Data_A6.xlsx"
#df1.to_excel(excel_file_path, index=False)

In [22]:
df1

,Name,Address,Other Info,Price
0,"Sovereign Gate, 18-20 Kew Road, Richmond TW9 2NA",OFFICE TO LET RICHMOND / SERVICED OFFICE DESKS,Office Serviced Desks to let Richmond\nDesk ra...,Desk rates from £485 – £500 per desk
1,"179 High Street, Hampton Hill TW12 1NL",NOW LET TO ESTABLISHED SUPERMARKET CHAIN,"Hampton Hill, London\nGross Internal Area: 4,2...","Gross Internal Area: 4,209 sq. ft. (391 sq. m)..."
2,"28 York Street, Twickenham TW1 3LJ",﻿,FREEHOLD (3 new flats & 1 shop)\nUnconditional...,"Unconditional Offers of £1,390,000"
